In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re

In [20]:
url = 'https://www.j-archive.com/listseasons.php'
r = requests.get(url)
soup = BeautifulSoup(r.content)
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">\n<html xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <title>\n   J! Archive - Season list\n  </title>\n  <link href="http://www.j-archive.com/styles.css" rel="styleSheet" type="text/css"/>\n  <link href="http://www.j-archive.com/favicon.ico" rel="shortcut icon"/>\n </head>\n <body>\n  <div id="navbar">\n   <span id="navbarlogo">\n    <a href="http://www.j-archive.com">\n     <img alt="J! Archive" height="22" src="http://www.j-archive.com/j-a.gif" width="100"/>\n    </a>\n   </span>\n   <span id="navbartext">\n    <a href="http://www.j-archive.com/showseason.php?season=35">\n     [current season]\n    </a>\n    <a href="http://www.j-archive.com/showseason.php?season=34">\n     [last season]\n    </a>\n    <a href="http://www.j-archive.com/listseasons.php">\n     [all seasons]\n    </a>\n    <a href="http://www.j-archive.com/listprizes.php">\n     [prizes]\n    </a>\n    <

In [21]:
# Get links for seasons
season_links = []
for link in soup.find_all('a'):
#     print(link.get('href'))
    if 'http' not in link.get('href'):
        season_links.append(link.get('href'))
season_links

['showseason.php?season=35',
 'showseason.php?season=34',
 'showseason.php?season=33',
 'showseason.php?season=32',
 'showseason.php?season=31',
 'showseason.php?season=30',
 'showseason.php?season=29',
 'showseason.php?season=28',
 'showseason.php?season=27',
 'showseason.php?season=26',
 'showseason.php?season=25',
 'showseason.php?season=24',
 'showseason.php?season=23',
 'showseason.php?season=22',
 'showseason.php?season=21',
 'showseason.php?season=20',
 'showseason.php?season=19',
 'showseason.php?season=18',
 'showseason.php?season=17',
 'showseason.php?season=16',
 'showseason.php?season=15',
 'showseason.php?season=14',
 'showseason.php?season=13',
 'showseason.php?season=12',
 'showseason.php?season=11',
 'showseason.php?season=10',
 'showseason.php?season=9',
 'showseason.php?season=8',
 'showseason.php?season=7',
 'showseason.php?season=superjeopardy',
 'showseason.php?season=6',
 'showseason.php?season=5',
 'showseason.php?season=4',
 'showseason.php?season=3',
 'showseas

In [43]:
# season_url = 'https://www.j-archive.com/' + season_links[0]
# print(season_url)
relevant_seasons = []
for i in season_links:
    season_url = 'https://www.j-archive.com/' + i
    r=requests.get(season_url)
    soup = BeautifulSoup(r.content)
    soup.prettify()
    for contestants in soup.find_all('td', {'valign': 'top'}):
        if 'Ken Jennings' in contestants.text or 'James Holzhauer' in contestants.text:
            relevant_seasons.append(i)
relevant_seasons = set(relevant_seasons)

In [83]:
relevant_seasons
# For now just take season games
relevant_seasons = ['showseason.php?season=20', 'showseason.php?season=21','showseason.php?season=35']

In [133]:
relevant_episodes = []
for i in relevant_seasons:
    contestant_index = []
    season_url = 'https://www.j-archive.com/' + i
    r=requests.get(season_url)
    soup = BeautifulSoup(r.content)
    soup.prettify()
    for num, td in enumerate(soup.find_all('td', {'valign': 'top'})):
        if ('Ken Jennings game' in td.text) or ('James Holzhauer game' in td.text):
#         if ('James Holzhauer game' in td.text):
            contestant_index.append(num)
    link_index = [x - 2 for x in contestant_index]  
    for num, contestants in enumerate(soup.find_all('td', {'valign': 'top'})):
        if num in link_index:
            relevant_episodes.append(contestants.find('a').get('href'))


104

In [ ]:
# Download HTML for an individual show to test code on
episode_dict = {}
count = 1
for num, show_url in enumerate(relevant_episodes):
    print(show_url)
    player_score_dict = {}
    r=requests.get(show_url)
    soup = BeautifulSoup(r.content)
    contestant_names = []
    for i in soup.find_all(class_ = 'score_player_nickname'):
        contestant_names.append(i.text)
    contestant_names = contestant_names[:3]
    contestant_scores = []
    try: 
        contestant_scores.append(soup.find('h3', text = 'Final scores:').find_next(class_='score_positive').text)
        contestant_scores.append(soup.find('h3', text = 'Final scores:').find_next(class_='score_positive').find_next(class_='score_positive').text)
        contestant_scores.append(soup.find('h3', text = 'Final scores:').find_next(class_='score_positive').find_next(class_='score_positive').find_next(class_='score_positive').text)
    except:
        pass
    results_dict = dict(zip(contestant_names, contestant_scores))
    episode_dict[count] = results_dict
    print(results_dict)
    count+= 1







http://www.j-archive.com/showgame.php?game_id=920
{'Ken': '$75,000', 'Sarah': '$1,200', 'Kristine': '$1,800'}
http://www.j-archive.com/showgame.php?game_id=2732
{'Ken': '$52,000', 'Michael': '$4,001', 'Denele': '$1,200'}
http://www.j-archive.com/showgame.php?game_id=2731
{'Ken': '$30,000', 'Liese': '$0', 'Art': '$8,800'}
http://www.j-archive.com/showgame.php?game_id=2730
{'Ken': '$29,200', 'Christopher': '$6,399', 'Susan': '$200'}
http://www.j-archive.com/showgame.php?game_id=2729
{'Ken': '$35,000', 'Beverly': '$6,800', 'Tim': '$7,900'}
http://www.j-archive.com/showgame.php?game_id=2485
